In [104]:
# импорт библиотек

import pandas as pd 

import re # регулярки

# прогресс-бар
from tqdm.notebook import trange, tqdm

In [2]:
# путь к файлам

file_path = 'C:/01_Projects/02_elk_accidents/'

In [7]:
# выгружаю файл в датафрейм

work = pd.read_csv(file_path + 'elks.csv')

In [8]:
# смотрю случайные строки

work.sample(3)

,act_name,court_info,act_text,act_url,instance,queries
2645,Решение № 2-141/2016 от 27 июня 2016 г. по дел...,Саянский районный суд (Красноярский край) - Гр...,Решение № 2-141/2016 от 27 июня 2016 г. по дел...,https://sudact.ru/regular/doc/18jP0rcthFVl/,10,['столкновение с медведем']
311,Постановление № 5-27/2013 от 23 октября 2013 г.,Якшур-Бодьинский районный суд (Удмуртская Респ...,Постановление № 5-27/2013 от 23 октября 2013 г...,https://sudact.ru/regular/doc/67zn4rGDv5mT/,10,"['наезд на лося', 'столкновение с лосем', 'авт..."
224,Постановление № 5-1041/2015 от 6 ноября 2015 г...,Рыбинский городской суд (Ярославская область) ...,Постановление № 5-1041/2015 от 6 ноября 2015 г...,https://sudact.ru/regular/doc/gGILJdXa5ixR/,10,['наезд на лося']


In [9]:
# количество строк и столбцов

work.shape

(3165, 6)

In [37]:
# меняю типы столбцов

work = work.astype({'instance': 'int32', 'text_len': 'int32'})

In [38]:
# типы данных

work.dtypes

act_name      object
court_info    object
act_text      object
act_url       object
instance       int32
queries       object
text_len       int32
dtype: object

In [35]:
# проверка на пустые значения

work.isnull().sum()

act_name      0
court_info    0
act_text      0
act_url       0
instance      0
queries       0
text_len      0
dtype: int64

### Чищу тексты дел

In [103]:
work.columns

Index(['act_name', 'court_info', 'act_text', 'act_url', 'instance', 'queries',
       'text_len', 'act_date'],
      dtype='object')

In [105]:

# список столбцов со строковыми значениями
colum_str_list = ['act_name', 'court_info', 'act_text', 'act_url', 'act_date']

for column in tqdm(colum_str_list):
    work[column] = work[column].str.replace('ё', 'е')
    work[column] = work[column].str.replace('\s+', ' ', regex=True)    

  0%|          | 0/5 [00:00<?, ?it/s]

In [106]:
# считаю длину текста судебных решений

work['text_len'] = work['act_text'].apply(lambda text: len(text))

In [108]:
# дата решения

work['act_date'] = work['act_name'].str.replace(r'.+ от ', ' ', regex=True)
work['act_date'] = work['act_date'].str.replace(r' г\. .+', ' ', regex=True)
work['act_date'] = work['act_date'].str.replace(r' г\.', ' ', regex=True)

In [109]:
work.sample(3)

,act_name,court_info,act_text,act_url,instance,queries,text_len,act_date
371,Приговор № 1-412/2011 от 2 августа 2011 г. по ...,Головинский районный суд (Город Москва) - Угол...,Приговор № 1-412/2011 от 2 августа 2011 г. по ...,https://sudact.ru/regular/doc/k1WSfnbcrAlC/,10,"['наезд на лося', 'столкновение с лосем', 'авт...",23231,2 августа 2011
1311,Апелляционное постановление № 22-2221/2019 от ...,Верховный Суд Удмуртской Республики (Удмуртска...,Апелляционное постановление № 22-2221/2019 от ...,https://sudact.ru/regular/doc/8diuLPVqprLQ/,20,['автомобиль лось'],19074,26 декабря 2019
2860,Приговор № 1-96/2014 от 14 октября 2014 г. по ...,Солецкий районный суд (Новгородская область) -...,Приговор № 1-96/2014 от 14 октября 2014 г. по ...,https://sudact.ru/regular/doc/trNPV4jtevL1/,10,['автомобиль лось'],165067,14 октября 2014


In [111]:
temp = work.act_url.value_counts().reset_index()

In [115]:
temp = temp[temp.act_url > 1]['index'].to_list()

In [118]:
temp_work = pd.DataFrame()

for i in temp:
    temp_work = temp_work.append(work[work.act_url == i])
    

In [119]:
temp_work

,act_name,court_info,act_text,act_url,instance,queries,text_len,act_date
0,Решение № 2-141/2018 2-141/2018 (2-5541/2017;)...,Ленинский районный суд г. Кирова (Кировская об...,Решение № 2-141/2018 2-141/2018 (2-5541/2017;)...,https://sudact.ru/regular/doc/OEVBjmP8HEx9/,10,"['наезд на лося', 'столкновение с лосем', 'авт...",21533,12 февраля 2018
501,Решение № 2-141/2018 2-141/2018 (2-5541/2017;)...,Ленинский районный суд г. Кирова (Кировская об...,Решение № 2-141/2018 2-141/2018 (2-5541/2017;)...,https://sudact.ru/regular/doc/OEVBjmP8HEx9/,10,"['наезд на лося', 'столкновение с лосем', 'авт...",21541,12 февраля 2018
1873,Решение № 2-429/2019 2-429/2019~М-389/2019 М-3...,Далматовский районный суд (Курганская область)...,Решение № 2-429/2019 2-429/2019~М-389/2019 М-3...,https://sudact.ru/regular/doc/XvMg57oaJQbZ/,10,['автомобиль лось'],20936,26 декабря 2019
2834,Решение № 2-429/2019 2-429/2019~М-389/2019 М-3...,Далматовский районный суд (Курганская область)...,Решение № 2-429/2019 2-429/2019~М-389/2019 М-3...,https://sudact.ru/regular/doc/XvMg57oaJQbZ/,10,['автомобиль лось'],20944,26 декабря 2019
1702,Решение № 2-140/2012 от 29 марта 2012 г. по де...,Центральный районный суд г. Кемерово (Кемеровс...,Решение № 2-140/2012 от 29 марта 2012 г. по де...,https://sudact.ru/regular/doc/r9J1eLvStU68/,10,['автомобиль лось'],21537,29 марта 2012
1703,Решение № 2-140/2012 от 29 марта 2012 г. по де...,Центральный районный суд г. Кемерово (Кемеровс...,Решение № 2-140/2012 от 29 марта 2012 г. по де...,https://sudact.ru/regular/doc/r9J1eLvStU68/,10,['автомобиль лось'],21529,29 марта 2012
472,Постановление № 10-3-2014 10-3/2014 от 26 мая ...,Ижморский районный суд (Кемеровская область) -...,Постановление № 10-3-2014 10-3/2014 от 26 мая ...,https://sudact.ru/regular/doc/PIixYwrlhxml/,20,"['наезд на лося', 'автомобиль лось']",8751,26 мая 2014
502,Постановление № 10-3-2014 10-3/2014 от 26 мая ...,Ижморский районный суд (Кемеровская область) -...,Постановление № 10-3-2014 10-3/2014 от 26 мая ...,https://sudact.ru/regular/doc/PIixYwrlhxml/,20,"['наезд на лося', 'автомобиль лось']",8743,26 мая 2014


In [76]:
work_temp = work[~(
    
    (work.act_text.str.contains(' лося', case=False)) |
    (work.act_text.str.contains(' лосем', case=False)) |
    (work.act_text.str.contains(' лосенка', case=False)) |
    (work.act_text.str.contains(' лосих', case=False)) |
    (work.act_text.str.contains(' лось', case=False)) |
    (work.act_text.str.contains(' медвед', case=False))
    
    
)]

In [77]:
work_temp

,act_name,court_info,act_text,act_url,instance,queries,text_len
12,Решение № 2-225/2017 2-225/2017(2-9924/2016;)~...,Череповецкий городской суд (Вологодская област...,Решение № 2-225/2017 2-225/2017(2-9924/2016;)~...,https://sudact.ru/regular/doc/NQDG3zkUu31v/,10,"['наезд на лося', 'столкновение с лосем', 'авт...",16031
91,Постановление № 5-2350/2020 от 17 ноября 2020 ...,Псковский городской суд (Псковская область) - ...,Постановление № 5-2350/2020 от 17 ноября 2020 ...,https://sudact.ru/regular/doc/sGp5SKGtHe5t/,10,"['наезд на лося', 'столкновение с лосем', 'авт...",7943
99,Постановление № 5-25/2020 от 17 июля 2020 г. п...,Тобольский районный суд (Тюменская область) - ...,Постановление № 5-25/2020 от 17 июля 2020 г. п...,https://sudact.ru/regular/doc/zJuDnmbRnNrK/,10,"['наезд на лося', 'столкновение с лосем', 'авт...",12437
103,Постановление № 5-40/2020 от 27 мая 2020 г. по...,Ржевский городской суд (Тверская область) - Ад...,Постановление № 5-40/2020 от 27 мая 2020 г. по...,https://sudact.ru/regular/doc/Xge5yWFLVWg9/,10,"['наезд на лося', 'автомобиль лось']",16054
104,Решение № 2-274/2020 2-274/2020~М-164/2020 М-1...,Красноперекопский районный суд (Республика Кры...,Решение № 2-274/2020 2-274/2020~М-164/2020 М-1...,https://sudact.ru/regular/doc/2nwgFMJ287Zv/,10,"['наезд на лося', 'автомобиль лось']",19882
...,...,...,...,...,...,...,...
3160,Приговор № 1-183/2013 от 27 декабря 2013 г. по...,Ефремовский районный суд (Тульская область) - ...,Приговор № 1-183/2013 от 27 декабря 2013 г. по...,https://sudact.ru/regular/doc/U6nMLjGbk8JQ/,10,['автомобиль лось'],118227
3161,Приговор № 1-367/2013 от 26 декабря 2013 г. по...,Сарапульский городской суд (Удмуртская Республ...,Приговор № 1-367/2013 от 26 декабря 2013 г. по...,https://sudact.ru/regular/doc/nDm0bdU62Fv/,10,['автомобиль лось'],37245
3162,Приговор № 1-171/2013 от 24 декабря 2013 г. по...,Донецкий городской суд (Ростовская область) - ...,Приговор № 1-171/2013 от 24 декабря 2013 г. по...,https://sudact.ru/regular/doc/lf9U6Ia4gxWv/,10,['автомобиль лось'],53736
3163,Приговор № 1-298/2013 от 20 декабря 2013 г. по...,Пятигорский городской суд (Ставропольский край...,Приговор № 1-298/2013 от 20 декабря 2013 г. по...,https://sudact.ru/regular/doc/2oZ4qZ0LTZUu/,10,['автомобиль лось'],43707


In [93]:
work.sample(3)

,act_name,court_info,act_text,act_url,instance,queries,text_len,act_date
1207,Кассационное определение от 28 ноября 2012 г.,Кемеровский областной суд (Кемеровская область...,Кассационное определение от 28 ноября 2012 г.\...,https://sudact.ru/regular/doc/aaNmUlF7rQxu/,30,['автомобиль лось'],7451,28 ноября 2012
711,Решение № 2-150/2012 2-150/2012~М-128/2012 М-1...,Жуковский районный суд (Калужская область) - Г...,Решение № 2-150/2012 2-150/2012~М-128/2012 М-1...,https://sudact.ru/regular/doc/6CBIfN10X3rw/,10,"['столкновение с лосем', 'автомобиль лось']",9948,11 мая 2012
3085,Решение № 2-4473/2016 2-4473/2016~М-5992/2016 ...,Октябрьский районный суд г. Ростова-на-Дону (Р...,Решение № 2-4473/2016 2-4473/2016~М-5992/2016 ...,https://sudact.ru/regular/doc/rKNyqjeIqhZb/,10,['автомобиль лось'],25227,27 октября 2016


In [94]:
work.to_excel(file_path + 'temp_666.xlsx')

In [2]:
# для визуализации
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# убирает  переводы строки

df.column = df.column.str.replace('\n', ' ')

In [3]:
# статистических

import statistics

%matplotlib inline

In [ ]:
[i for i in list(students_df) if 'score' in i]

In [ ]:
[i for i in students_df.columns if 'score' in i]

In [ ]:
students_df.filter(like='score')

In [ ]:
# axis=0 - строки (в индексе), axis=1 - столбцы 

students_df.filter(like='1', axis=0)

In [ ]:
(
    students_df.groupby('gender', as_index=False)
    .agg({'math_score': 'mean'})
    .rename(columns={'math_score': 'math_score_mean'})
)

In [ ]:
# доля пропусков в данных

round(titanic_data.isna().mean() * 100, 2)